<h1>Loading Data from the University SFTP</h1>
<p>Please not that for this script to run you need to setup you password inside a .env file. To that file write PASSWORD=YOUR_PASSORD. You must also ensure your environment has the correct packages to handle sftp connection and python-dotenv</P>

In [1]:
import pysftp
import os 
import numpy as np
import pandas as pd
import numpy as np

from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

load_dotenv() 

Hostname = "eepc-0184.bath.ac.uk"
Username = "jmdomd20"
Password = os.environ.get('PASSWORD')

/Users/joaojesus/opt/anaconda3/envs/geo_plotting/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# =============================================================================
# Downloading data from sftp
# Parameters: 
# Grid Size: 40km
# Source: AIRS
# Hemisphere: Northern
# Data Field: tp, bg, a, ha, k, l, m, mfx, mfy
# Daytime: Night
# Altitude: 1 (30km)
# =============================================================================

grid = map_grid(grid_size='40', source='airs')
wizard = data_wizard()

download_flag = True
if download_flag:
    range_ = range(2016,2022+1)
    for year in range_:
        with pysftp.Connection(host=Hostname, username=Username, password=Password) as sftp:
            print("Connection successfully established ... ")
            # Switch to a remote directory
            sftp.cwd(f'/data3/AIRS/Climatology/40km_grid/{year}')

            # Obtain structure of the remote directory '/opt'
            directory_structure = sftp.listdir_attr()

            # # Print data
            # for attr in directory_structure:
            #     print(attr.filename, attr)
            
            for i in range(len(directory_structure)):
                    file_name = directory_structure[i].filename

                    year = int(file_name[:4])
                    month = int(file_name[4:6])
                    day = int(file_name[6:8])
                    date = f"{year}-{month}-{day}"
                    date = datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d')

                    print(f"downloading.... {file_name} for this date: {date}")
                    sftp.get(file_name) 

                    print('download complete')

                    matlab = matlab_reader(file_name)

                    data_tp = matlab.select(hemisphere= 'nh',data_field='tp',daytime='Night',altitude=1)   
                    data_bg = matlab.select(hemisphere= 'nh',data_field='bg',daytime='Night',altitude=1)   
                    data_a = matlab.select(hemisphere= 'nh',data_field='A',daytime='Night',altitude=1)   
                    data_ha = matlab.select(hemisphere= 'nh',data_field='HA',daytime='Night',altitude=1)   
                    data_k = matlab.select(hemisphere= 'nh',data_field='k',daytime='Night',altitude=1)   
                    data_l = matlab.select(hemisphere= 'nh',data_field='l',daytime='Night',altitude=1)   
                    data_m = matlab.select(hemisphere= 'nh',data_field='m',daytime='Night',altitude=1)   
                    data_mfx = matlab.select(hemisphere= 'nh',data_field='mfx',daytime='Night',altitude=1)   
                    data_mfy = matlab.select(hemisphere= 'nh',data_field='mfy',daytime='Night',altitude=1)   

                    select_area_indexer = grid.select_area_indexer(min_lat= 45,max_lat= 90,min_lon= -180,max_lon= 180)

                    data_tp = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_tp)
                    data_bg = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_bg)
                    data_a = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_a) 
                    data_ha = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_ha)   
                    data_k = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_k)
                    data_l = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_l)
                    data_m = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_m)
                    data_mfx = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_mfx) 
                    data_mfy = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_mfy) 

                    # np.savez_compressed(f"processed_airs_sftp/tp/{date}_tp.npz", data_tp)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/tp/{date}_tp.npz", data_tp)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/bg/{date}_bg.npz", data_bg)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/A/{date}_a.npz", data_a)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/ha/{date}_ha.npz", data_ha)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/k/{date}_k.npz", data_k)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/l/{date}_l.npz", data_l)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/m/{date}_m.npz", data_m)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/mfx/{date}_mfx.npz", data_mfx)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/30km/mfy/{date}_mfy.npz", data_mfy)

                    print('delete file')
                    # print(100*(i+1/(len(range_)*len(directory_structure))),' % complete')
                    os.remove(file_name)




Connection successfully established ... 
downloading.... 20160101_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-01
download complete
delete file
downloading.... 20160102_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-02
download complete
delete file
downloading.... 20160103_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-03
download complete
delete file
downloading.... 20160104_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-04
download complete
delete file
downloading.... 20160105_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-05
download complete
delete file
downloading.... 20160106_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-06
download complete
delete file
downloading.... 20160107_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-07
download complete
delete file
downloading.... 20160108_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-08
download complete
delete file
downloading.... 20160109_AIRS_3DST-1_40km_grid.mat for this date: 2016-01-09
download complete
delete f

In [ ]:
# =============================================================================
# Downloading data from sftp
# Parameters: 
# Grid Size: 40km
# Source: AIRS
# Hemisphere: Northern
# Data Field: tp, bg, a, ha, k, l, m, mfx, mfy
# Daytime: Night
# Altitude: 4 (48km)
# =============================================================================

grid = map_grid(grid_size='40', source='airs')
wizard = data_wizard()

download_flag = True
if download_flag:
    range_ = range(2009,2009+1)
    for year in range_:
        with pysftp.Connection(host=Hostname, username=Username, password=Password) as sftp:
            print("Connection successfully established ... ")
            # Switch to a remote directory
            sftp.cwd(f'/data3/AIRS/Climatology/40km_grid/{year}')

            # Obtain structure of the remote directory '/opt'
            directory_structure = sftp.listdir_attr()

            # # Print data
            # for attr in directory_structure:
            #     print(attr.filename, attr)
            
            for i in range(len(directory_structure)):
                    file_name = directory_structure[i].filename

                    year = int(file_name[:4])
                    month = int(file_name[4:6])
                    day = int(file_name[6:8])
                    date = f"{year}-{month}-{day}"
                    date = datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d')

                    print(f"downloading.... {file_name} for this date: {date}")
                    sftp.get(file_name) 

                    print('download complete')

                    matlab = matlab_reader(file_name)

                    data_bg = matlab.select(hemisphere= 'nh',data_field='bg',daytime='night',altitude=4)   
                    data_a = matlab.select(hemisphere= 'nh',data_field='a',daytime='night',altitude=4)   
                    data_ha = matlab.select(hemisphere= 'nh',data_field='ha',daytime='night',altitude=4)   
                    data_k = matlab.select(hemisphere= 'nh',data_field='k',daytime='night',altitude=4)   
                    data_l = matlab.select(hemisphere= 'nh',data_field='l',daytime='night',altitude=4)   
                    data_m = matlab.select(hemisphere= 'nh',data_field='m',daytime='night',altitude=4)   
                    data_mfx = matlab.select(hemisphere= 'nh',data_field='mfx',daytime='night',altitude=4)   
                    data_mfy = matlab.select(hemisphere= 'nh',data_field='mfy',daytime='night',altitude=4)   

                    select_area_indexer = grid.select_area_indexer(min_lat= 55,max_lat= 90,min_lon= -180,max_lon= 180)

                    data_bg = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_bg)
                    data_a = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_a) 
                    data_ha = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_ha)   
                    data_k = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_k)
                    data_l = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_l)
                    data_m = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_m)
                    data_mfx = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_mfx) 
                    data_mfy = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data_mfy) 

                    # np.savez_compressed(f"processed_airs_sftp/tp/{date}_tp.npz", data_tp)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/bg/{date}_bg.npz", data_bg)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/A/{date}_a.npz", data_a)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/ha/{date}_ha.npz", data_ha)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/k/{date}_k.npz", data_k)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/l/{date}_l.npz", data_l)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/m/{date}_m.npz", data_m)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/mfx/{date}_mfx.npz", data_mfx)
                    np.savez_compressed(f"processed_airs_sftp/40_km_grid/48km/mfy/{date}_mfy.npz", data_mfy)

                    print('delete file')
                    # print(100*(i+1/(len(range_)*len(directory_structure))),' % complete')
                    os.remove(file_name)


